In [1]:
import sympy as sp
import numpy as np
import math
from scipy.optimize import brentq

In [2]:
h_num = 35.0
H_num = 150.0
D_num = 8.0
t_num = 0.1

M_num = 1.0e6
J_num = 1.6e9
c_num = 30e3

E_num = 210e9
rho_num = 7850.0
rho_f_num = 1025.0
Ca_num = 1.0
T_num = 0.0

Di_num = D_num - 2.0*t_num
A_num = math.pi/4.0 * (D_num**2 - Di_num**2)
I_num = math.pi/64.0 * (D_num**4 - Di_num**4)

In [3]:
# -----------------------------
# Symbols
# -----------------------------
z, w = sp.symbols('z wega', real=True)
E, I, T = sp.symbols('E I T', positive=True, real=True)
rho, A = sp.symbols('rho A', positive=True, real=True)
rho_f, Ca, D = sp.symbols('rho_f Ca D', positive=True, real=True)
h, H = sp.symbols('h H', positive=True, real=True)
J, Mtip, c = sp.symbols('J M c', real=True)  
EI = E*I

# Effective mass per length (submerged) and (above water)
m1 = rho*A + rho_f*Ca*sp.pi*D**2/4   # submerged
m2 = rho*A                            # above water

# Spatial ODE parameters
beta1_4 = (w**2 * m1) / EI
beta2_4 = (w**2 * m2) / EI

# Define alpha^2 and mu^2 exactly as in your derivation
alpha1_sq = (-T/EI + sp.sqrt((T/EI)**2 + 4*beta1_4))/2
mu1_sq    = ( T/EI + sp.sqrt((T/EI)**2 + 4*beta1_4))/2

alpha2_sq = (-T/EI + sp.sqrt((T/EI)**2 + 4*beta2_4))/2
mu2_sq    = ( T/EI + sp.sqrt((T/EI)**2 + 4*beta2_4))/2

alpha1, mu1 = sp.sqrt(alpha1_sq), sp.sqrt(mu1_sq)
alpha2, mu2 = sp.sqrt(alpha2_sq), sp.sqrt(mu2_sq)

# -----------------------------
# General solutions W1(z), W2(z)
# -----------------------------
A1,B1,C1,D1 = sp.symbols('A1 B1 C1 D1')
A2,B2,C2,D2 = sp.symbols('A2 B2 C2 D2')

W1 = A1*sp.cosh(alpha1*z) + B1*sp.sinh(alpha1*z) + C1*sp.cos(mu1*z) + D1*sp.sin(mu1*z)
W2 = A2*sp.cosh(alpha2*z) + B2*sp.sinh(alpha2*z) + C2*sp.cos(mu2*z) + D2*sp.sin(mu2*z)

# Derivatives you will need for beam BCs
W1_1 = sp.diff(W1, z)
W1_2 = sp.diff(W1, z, 2)
W1_3 = sp.diff(W1, z, 3)

W2_1 = sp.diff(W2, z)
W2_2 = sp.diff(W2, z, 2)
W2_3 = sp.diff(W2, z, 3)

# Beam resultants (Euler-Bernoulli with axial tension T):
# Mwent:   M = EI * W''
# Shear:    V = EI * W''' + T * W'
M1 = EI*W1_2
V1 = EI*W1_3 + T*W1_1

M2 = EI*W2_2
V2 = EI*W2_3 + T*W2_1



In [8]:
# 8 boundary/interface equations (LHS = 0 vorm)
eq1 = W1.subs(z, -h)                        # w1(-h)=0
eq2 = W1_1.subs(z, -h)                      # w1'(-h)=0

eq3 = W1.subs(z, 0)  - W2.subs(z, 0)        # w1(0)=w2(0)
eq4 = W1_1.subs(z, 0) - W2_1.subs(z, 0)     # w1'(0)=w2'(0)
eq5 = W1_2.subs(z, 0) - W2_2.subs(z, 0)     # w1''(0)=w2''(0)
eq6 = W1_3.subs(z, 0) - W2_3.subs(z, 0)     # w1'''(0)=w2'''(0)

# Top BCs at z=H with harmonic time dependence w(z,t)=W(z)*exp(i*w*t)
# EI w2_zz + J w2_ttz = 0  -> EI*W2'' - J*w^2*W2' = 0
eq7 = EI*W2_2.subs(z, H) - J*w**2*W2_1.subs(z, H)

# EI w2_zzz + M w2_tt + c w2_t = 0
# -> EI*W2''' - M*w^2*W2 + i*c*w*W2 = 0
eq8 = EI*W2_3.subs(z, H) - Mtip*w**2*W2.subs(z, H) + sp.I*c*w*W2.subs(z, H)

# Collect them
eqs = [eq1,eq2,eq3,eq4,eq5,eq6,eq7,eq8]

# Unknown constants vector
Cvec = sp.Matrix([A1,B1,C1,D1,A2,B2,C2,D2])

# Build K(w) by extracting coefficients of the constants
K = sp.Matrix([[sp.diff(expr, cst) for cst in Cvec] for expr in eqs])
print("K(w) matrix:")
print(K)

K(w) matrix:
Matrix([[cosh(h*sqrt(sqrt(4*wega**2*(A*rho + pi*Ca*D**2*rho_f/4)/(E*I) + T**2/(E**2*I**2))/2 - T/(2*E*I))), -sinh(h*sqrt(sqrt(4*wega**2*(A*rho + pi*Ca*D**2*rho_f/4)/(E*I) + T**2/(E**2*I**2))/2 - T/(2*E*I))), cos(h*sqrt(sqrt(4*wega**2*(A*rho + pi*Ca*D**2*rho_f/4)/(E*I) + T**2/(E**2*I**2))/2 + T/(2*E*I))), -sin(h*sqrt(sqrt(4*wega**2*(A*rho + pi*Ca*D**2*rho_f/4)/(E*I) + T**2/(E**2*I**2))/2 + T/(2*E*I))), 0, 0, 0, 0], [-sqrt(sqrt(4*wega**2*(A*rho + pi*Ca*D**2*rho_f/4)/(E*I) + T**2/(E**2*I**2))/2 - T/(2*E*I))*sinh(h*sqrt(sqrt(4*wega**2*(A*rho + pi*Ca*D**2*rho_f/4)/(E*I) + T**2/(E**2*I**2))/2 - T/(2*E*I))), sqrt(sqrt(4*wega**2*(A*rho + pi*Ca*D**2*rho_f/4)/(E*I) + T**2/(E**2*I**2))/2 - T/(2*E*I))*cosh(h*sqrt(sqrt(4*wega**2*(A*rho + pi*Ca*D**2*rho_f/4)/(E*I) + T**2/(E**2*I**2))/2 - T/(2*E*I))), sqrt(sqrt(4*wega**2*(A*rho + pi*Ca*D**2*rho_f/4)/(E*I) + T**2/(E**2*I**2))/2 + T/(2*E*I))*sin(h*sqrt(sqrt(4*wega**2*(A*rho + pi*Ca*D**2*rho_f/4)/(E*I) + T**2/(E**2*I**2))/2 + T/(2*E*I))), s

In [5]:
vals = {
    E: E_num,
    I: I_num,
    T: T_num,
    rho: rho_num,
    A: A_num,
    rho_f: rho_f_num,
    Ca: Ca_num,
    D: D_num,
    h: h_num,
    H: H_num,
    J: J_num,
    Mtip: M_num,
    c: 0.0
}

In [6]:
# Numerieke determinantfunctie (snel)
K_w = K.subs(vals)
K_func = sp.lambdify(w, K_w, modules='numpy')

def detK_numeric(omega):
    K_eval = np.array(K_func(float(omega)), dtype=np.complex128)
    return np.linalg.det(K_eval)

# Voor ongedempte case (c=0) is det(K) reëel op numerieke ruis na
f = lambda om: float(np.real(detK_numeric(om)))

# Zoekgebied en resolutie (pas aan indien nodig)
omega_min, omega_max = 0.0001, 5.0
n_scan = 2000
omega_grid = np.linspace(omega_min, omega_max, n_scan)
f_grid = np.array([f(om) for om in omega_grid])

# Bracketing op tekenwisselingen + root finding
roots = []
for i in range(len(omega_grid) - 1):
    a, b = omega_grid[i], omega_grid[i + 1]
    fa, fb = f_grid[i], f_grid[i + 1]

    if np.isfinite(fa) and abs(fa) < 1e-8:
        roots.append(a)
        continue

    if np.isfinite(fa) and np.isfinite(fb) and fa * fb < 0:
        try:
            r = brentq(f, a, b, maxiter=200)
            roots.append(r)
        except ValueError:
            pass

# Dubbels verwijderen (door aangrenzende brackets)
roots = np.array(sorted(roots))
if roots.size > 0:
    unique_roots = [roots[0]]
    for r in roots[1:]:
        if abs(r - unique_roots[-1]) > 1e-4:
            unique_roots.append(r)
    roots = np.array(unique_roots)

print('Gevonden omega [rad/s] waarvoor det(K)=0:')
print(roots)

# Controle: determinantwaarde op roots
for r in roots:
    print(f'omega={r:.6f}, det={detK_numeric(r):.20f}')

Gevonden omega [rad/s] waarvoor det(K)=0:
[1.00000000e-04 2.60120060e-03 5.10240120e-03 7.60360180e-03
 1.01048024e-02 1.26060030e-02 1.51072036e-02 1.76084042e-02
 3.94981577e+00]
omega=0.000100, det=0.00000000000000000000+0.00000000000000000000j
omega=0.002601, det=0.00000000000006270954+0.00000000000000000000j
omega=0.005102, det=0.00000000000357221793+0.00000000000000000000j
omega=0.007604, det=0.00000000003912066173+0.00000000000000000000j
omega=0.010105, det=0.00000000021550458382+0.00000000000000000000j
omega=0.012606, det=0.00000000081236916234+0.00000000000000000000j
omega=0.015107, det=0.00000000240653838100+0.00000000000000000000j
omega=0.017608, det=0.00000000603403012104+0.00000000000000000000j
omega=3.949816, det=0.00000000474378867984+0.00000000000000000000j
